In [1]:
# Section 1 - Imports
# -------------------

# imports
import datetime
import attotime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
from os.path import join
import copy

# DSO modules
from dsoclasses.doris.algorithms import beacon_nominal_frequency
from dsoclasses.rinex.doris.rinex import DorisRinex
from dsoclasses.time.pyattotime import at2pt, fsec2asec, to_attoseconds
from dsoclasses.orbits import sp3c, interpolator
from dsoclasses.geodesy import transformations
from dsoclasses.gnss import systems as gs
from dsoclasses.gnss import algorithms as alg
from dsoclasses.troposphere.vmf3 import SiteVmf3
from dsoclasses.sinex import sinex

In [2]:
# Section 2.1 – Define Files needed for analysis
# -----------------------------------------------

data_path = "/home/xanthos/Software/AcademicSoftware/data"
drinex = join(data_path, "s6arx24001.001")
dpod = join(data_path, "dpod2020_041.snx")
dpod_freq_corr = join(data_path, "dpod2020_041_freq_corr.txt")
dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")
vmf3_data = join(data_path, "y2024.vmf3_d")

In [3]:
# Section 2.2 - Load Orbit Data + Initialize Interpolator
# -------------------------------------------------------

intrp = interpolator.Sp3Interpolator.from_sp3(dsp3, ['L'], interval_in_sec=310, min_data_pts=10, itype='Barycentric')

In [4]:
# Section 2.3 - Intitialize DorisRinex and select a beacon (Dionysos/DIOB)
# ------------------------------------------------------------------------

rnx = DorisRinex(drinex)

site_name = 'DIOB'
result = sinex.extract_sinex_coordinates(dpod, [site_name], rnx.time_first_obs, True, dpod_freq_corr)
diob_xyz = np.array([result['DIOB']['X'], result['DIOB']['Y'], result['DIOB']['Z']])

# we are also going to need the geodetic coordinates of the site
lat, lon, hgt = transformations.car2ell(*diob_xyz)

In [5]:
# Section 2.4 - Initialize VMF3 from (DORIS) site data
# ----------------------------------------------------
vmf = SiteVmf3(vmf3_data, [site_name])

In [76]:
# Section 3.1 - Prerequisites & helper functions
# ----------------------------------------------

C = 299792458e0

# Ionosphere-free LC for phase observables
def getL3fun(nS1, nU2):
    gamma_sqrt = nS1 / nU2
    gamma = gamma_sqrt * gamma_sqrt
    l3_freq = (gamma * nS1 - gamma_sqrt * nU2) / (gamma - 1.)
    def l3(s1, u2):
        return (gamma * s1 - gamma_sqrt * u2) / (gamma - 1.), l3_freq
    return l3

# For DIOB ...
s1, u2 = beacon_nominal_frequency(rnx.kfactor(site_name))
l3 = getL3fun(s1, u2)

class L3Obs:
    def __init__(self, t, l3, sat_pos, dT, el, feN, frT, pass_nr):
        self.t = t
        self.val = l3
        self.rsat = sat_pos
        self.Tropo = dT
        self.el = el
        self.feN = feN
        self.frT = frT
        self.pass_nr = pass_nr
    def dTropo(self, zwd=None):
        zwd = self.Tropo['zwd'] if zwd is None else zwd
        d = self.Tropo
        return d["zhd"] * d["mfh"] + d["zwd"] * d["mfw"]

class DopplerObs:
    def __init__(self, l3t1, l3t2):
        self.l3t1 = copy.deepcopy(l3t1)
        self.l3t2 = copy.deepcopy(l3t2)
        assert self.l3t1.feN == self.l3t2.feN
        assert self.l3t1.pass_nr == self.l3t2.pass_nr
    def feN(self): return self.l3t1.feN
    def frT(self): return (self.l3t1.frT + self.l3t2.frT) / 2.
    def Ncount(self): return self.l3t2.val - self.l3t1.val
    def dt(self): # returns t2-t1 in [seconds]
        return float((self.l3t2.t-self.l3t1.t).total_nanoseconds()) * 1e-9
    def ttai(self): 
        return self.l3t1.t + (self.l3t2.t-self.l3t1.t) // 2
    def Ndoppler(self): return self.Ncount() / self.dt()
    def elevation(self): return (self.l3t1.el + self.l3t2.el) / 2.
    def pass_nr(self): return self.l3t1.pass_nr
    def r1(self, rsta): return np.linalg.norm(self.l3t1.rsat - rsta)
    def r2(self, rsta): return np.linalg.norm(self.l3t2.rsat - rsta)
    def rsat1(self): return self.l3t1.rsat
    def rsat2(self): return self.l3t2.rsat
    def observed_rr(self):
        return (C/self.feN()) * (self.feN() - self.frT() - self.Ndoppler()) # + (self.l3t2.dT - self.l3t1.dT)
    def computed_rr(self, rsta, df, zwd):
        dT = self.l3t2.dTropo(zwd) - self.l3t1.dTropo(zwd)
        return (self.r2(rsta) - self.r1(rsta)) / self.dt() + dT - (C/self.feN()) * (self.Ndoppler()+self.frT()) * df
    def ddzwd(self):
        return self.l3t2.Tropo['mfw'] - self.l3t1.Tropo['mfw']
    def apriori_zwd(self): return (self.l3t2.Tropo['zwd'] + self.l3t1.Tropo['zwd']) / 2.
    def set_pass_nr(self, nr):
        self.l3t1.pass_nr = nr
        self.l3t2.pass_nr = nr

In [85]:
# Section 3.2 - Initialize variables
# ----------------------------------

tprev   = None # previous (to current) observation epoch
L3_prev = None # previous L3 observation
CUTOFF_ANGLE = np.radians(7.) # Elevation cut-off angle
doppler = []    # Doppler observations collected
pass_nr = 0    # satellite pass number
tai_prev = None # time of previous obs, to count passes
passes_start_indexes = {} # start index in doppler list for every pass

In [86]:
# Section 4 – Preprocess Doppler observations (filter valid ones for LS)
# ----------------------------------------------------------------------

# Just checking that the above block is restarted before continuing ...
assert pass_nr == 0

# for every block in the RINEX file
for block in rnx:
    # for every beacon in the block
    for beacon, data in block:
        # match DIOB
        if beacon == rnx.name2id(site_name):
            
            # use the block-provided clock correction to get to (approximate) TAI
            # date(TAI) = epoch + receiver clock offset
            tai = block.t() + attotime.attotimedelta(nanoseconds=block.clock_offset() * 1e9)

            # see if we have a new pass
            if (tai_prev is None or (tai-tai_prev).total_seconds() > 600.) and len(doppler)!=0:
                pass_nr += 1
                passes_start_indexes[pass_nr] = len(doppler)
            tai_prev = tai
            
            # Satellite position at signal emission time
            satx, saty, satz, _, _ = alg.sat_at_emission_time(diob_xyz, tai, intrp, 'L40')
            rsat = np.array([satx, saty, satz])

            # Elevation and tropospheric delay
            _, _, el = transformations.azele(rsat, diob_xyz)
            if el >= CUTOFF_ANGLE:
                Tropo = vmf.tropo_dealy_info(site_name, lat, lon, el, at2pt(tai))
    
                # Phase measurements L3 combination
                L3, L3f = l3(data['L1']['value'], data['L2']['value'])
    
                # apply frequency offset - get true receiver frequency
                frT = L3f * (1 + data['F']['value'] * 1e-11)

                # form the new observation
                L3_cur = L3Obs(tai, L3, rsat, Tropo, el, L3f, frT, pass_nr)
    
                # if we have a previous measurement, within 10 sec ...
                if L3_prev is not None and (L3_cur.t-L3_prev.t).total_seconds() <= 10.:
                    doppler.append(DopplerObs(L3_prev, L3_cur))

                # prepare for next measurement
                L3_prev = L3_cur
                    
print(f"Number of Doppler observations extracted: {len(doppler)}")

Number of Doppler observations extracted: 965


In [87]:
# Section 4.1 - Examine passes
# ----------------------------

passes_info = {}
for data in doppler:
    ps = data.pass_nr()
    if ps in passes_info:
        passes_info[ps] += 1
    else:
        passes_info[ps] = 1
print('Number of data per pass:')
for k,v in passes_info.items():
    print(f'Pass {k} Data {v}')
for k,v in passes_start_indexes.items():
    print(f'Pass {k} start_index {v}')

Number of data per pass:
Pass 0 Data 153
Pass 1 Data 198
Pass 2 Data 118
Pass 3 Data 4
Pass 4 Data 142
Pass 5 Data 198
Pass 6 Data 152
Pass 1 start_index 153
Pass 2 start_index 351
Pass 3 start_index 469
Pass 4 start_index 473
Pass 5 start_index 615
Pass 6 start_index 813


In [88]:
MIN_DATA_PTS_IN_PASS = 10

def build_pass_starts(doppler):
    """Return {pass_id: start_index} from the doppler list."""
    starts = {}
    if not doppler:
        return starts
    cur = doppler[0].pass_nr()
    starts[cur] = 0
    for i in range(1, len(doppler)):
        pn = doppler[i].pass_nr()
        if pn != cur:
            starts[pn] = i
            cur = pn
    return starts

data_removed = True
while data_removed:
    data_removed = False  # reset each pass
    starts = build_pass_starts(doppler)
    if not starts:
        break

    pass_ids = sorted(starts.keys())
    for idx, pid in enumerate(pass_ids):
        start = starts[pid]
        end = starts[pass_ids[idx + 1]] if idx + 1 < len(pass_ids) else len(doppler)
        num_pts = end - start

        if num_pts < MIN_DATA_PTS_IN_PASS:
            print(f"Should remove pass {pid} from {start} to {end}")
            # remove the slice
            del doppler[start:end]
            # renumber subsequent entries
            for j in range(start, len(doppler)):
                if doppler[j].pass_nr() > pid:
                    doppler[j].set_pass_nr(doppler[j].pass_nr() - 1)
            data_removed = True
            break  # restart while-loop with rebuilt starts
        

Should remove pass 3 from 469 to 473


In [89]:
# Section 5 – Weighted Least Squares Solution
# --------------------------------------------

Np = pass_nr + 1 # Number of passes

# Initial guess: [x, y, z, Δf, ΔT_pass1, ΔT_pass2, ..., ΔT_passN]
x0 = np.zeros(3 + Np + Np) # parameter vector
x0[0:3] = diob_xyz         # start with nominal beacon position
# a-priori ZWD, per pass
x0[3+Np] = doppler[0].apriori_zwd()
for j in range(1, len(doppler)-1):
    if doppler[j].pass_nr() != doppler[j-1].pass_nr():
        x0[3+Np+doppler[j].pass_nr()] = doppler[j].apriori_zwd()
        print(f'-> adding ZWD for pass={doppler[j].pass_nr()}, j={j}')
        assert (3+Np+doppler[j].pass_nr()) < (3+Np+Np)
        
x = x0.copy()
# tai0 = doppler[0].ttai()

m = len(doppler)           # number of valid Doppler observations
J = np.zeros((m, len(x0))) # Jacobian matrix (design matrix)
dl = np.zeros(m)           # residual vector (observed - modeled)

sigma0 = .5                # sigma 0 in [m/sec]
elvs = np.array([obs.elevation() for obs in doppler]) # elevations arrays
sigma_i = sigma0 / np.sin(elvs)                      # per-observation stdev
sqrt_w = 1.0 / sigma_i                               # = sqrt(w_i)
W = sqrt_w**2                                        # = 1/sigma_i^2

LS_ITERATIONS = 1             # number of least squares iterations

for ls_iteration in range(LS_ITERATIONS):
    J = np.zeros((m, len(x)))
    dl = np.zeros(m)
    for i, obs in enumerate(doppler):
        dt = float((obs.ttai()-tai0).total_nanoseconds()) * 1e-9

        # Current parameter estimates
        r_beacon = x[0:3]               # beacon position estimate
        pass_id = obs.pass_nr()
        df      = x[3 + pass_id]        # frequency bias
        zwd     = x[3 + Np + pass_id]   # zwd
        
        # residual (observed - modeled), UNWEIGHTED
        dl[i] = obs.observed_rr() + obs.computed_rr(r_beacon, df, zwd)
        
        if abs(dl[i]) > 10.0:
            print(f"Skipping obs {i} due to large residual: {dl[i]:+.3f} m/s")
            sqrt_w[i] = 0.0
            continue

        # Jacobian Matrix
        J[i, 0:3] = (r_beacon - obs.rsat2()) / (obs.r2(r_beacon) * obs.dt()) - (r_beacon - obs.rsat1()) / (obs.r1(r_beacon) * obs.dt())
        J[i, 3 + pass_id] = +1.0
        J[i, 3 + Np + pass_id] = obs.ddzwd()
        
        
    Jw  = J * sqrt_w[:, None]      # each row scaled by sqrt(w_i)
    dlw = dl * sqrt_w              # same scaling for rhs

    # least squares solution
    dx, *_ = np.linalg.lstsq(Jw, dlw, rcond=None)
    x += dx

    # Posterior sigma of unit weight (scalar)
    dof = m - len(x)
    sigma_hat0 = np.sqrt(np.sum(dlw**2) / dof)   # == sqrt(sum(w*dl**2)/dof)

# de-weight residuals
dl_unweighted = np.divide(dlw, sqrt_w, out=np.zeros_like(dlw), where=sqrt_w!=0)

-> adding ZWD for pass=1, j=153
-> adding ZWD for pass=2, j=351
-> adding ZWD for pass=3, j=469
-> adding ZWD for pass=4, j=611
-> adding ZWD for pass=5, j=809


In [90]:
# Section 6.3 – Final parameter estimates and comparison to initial guess

final_xyz = x[0:3]
initial_xyz = x0[0:3]
delta_xyz = final_xyz - initial_xyz
delta_norm = np.linalg.norm(delta_xyz)

print("\n[RESULT] Beacon estimated position vs initial (XYZ):")
print(f"  X = {final_xyz[0]:+15.3f} m   | X_init = {initial_xyz[0]:+15.3f} m   | ΔX = {delta_xyz[0]:+15.3f} m")
print(f"  Y = {final_xyz[1]:+15.3f} m   | Y_init = {initial_xyz[1]:+15.3f} m   | ΔY = {delta_xyz[1]:+15.3f} m")
print(f"  Z = {final_xyz[2]:+15.3f} m   | Z_init = {initial_xyz[2]:+15.3f} m   | ΔZ = {delta_xyz[2]:+15.3f} m")

print(f"\n[RESULT] Total position error (‖Δr‖): {delta_norm:.3f} m")
print(f"\n[RESULT] Sigma0 A-Posteriori        : {sigma_hat0:.2f} [m/s]")

print("\n[RESULT] Per-Pass estimated values:")
for ps in range(Np):
    print(f"Pass Nr. {ps+1} dF={x[3+ps]:.6f} dT={x[3+Np+ps]:+.3f}")


[RESULT] Beacon estimated position vs initial (XYZ):
  X =    +4595209.971 m   | X_init =    +4595212.885 m   | ΔX =          -2.914 m
  Y =    +2039475.155 m   | Y_init =    +2039474.042 m   | ΔY =          +1.113 m
  Z =    +3912617.019 m   | Z_init =    +3912618.015 m   | ΔZ =          -0.996 m

[RESULT] Total position error (‖Δr‖): 3.275 m

[RESULT] Sigma0 A-Posteriori        : 2.57 [m/s]

[RESULT] Per-Pass estimated values:
Pass Nr. 1 dF=-2.942491 dT=+1.949
Pass Nr. 2 dF=-2.941694 dT=+2.044
Pass Nr. 3 dF=-2.944722 dT=+1.896
Pass Nr. 4 dF=-2.935110 dT=+1.928
Pass Nr. 5 dF=-2.946639 dT=+2.007
Pass Nr. 6 dF=-2.946342 dT=+2.038
Pass Nr. 7 dF=-0.000000 dT=+0.000
